In [0]:
import numpy as np
from numpy import dot, matrix, deg2rad, cross
from math import pi
import math
from math import cos, sin, acos, asin

input data: DH parameters of kuka iiwa7, initial theta angles, end effector position transformation matrix. 

In [0]:
# def r2d(x):
#   return x*180/pi

In [0]:
d_bs, d_se, d_ew, d_wf = 340, 400, 400, 126
d = np.array([d_bs, 0, d_se, 0, d_ew, 0, d_wf])
#theta = np.array([d2r(-115.8158), d2r(107.2721),d2r(0),d2r(75),d2r(-143.2641),d2r(25.8884),d2r(-90.9575)])
theta = np.array([deg2rad(30), deg2rad(-45),deg2rad(60),deg2rad(75),deg2rad(-20),deg2rad(95),deg2rad(-80)])
#theta = np.array([deg2rad(0), deg2rad(0),deg2rad(0),deg2rad(0),deg2rad(0),deg2rad(0),deg2rad(0)])
alpha = np.array([-pi/2, pi/2, pi/2, -pi/2, -pi/2, pi/2, 0])
a = np.array([0,0,0,0,0,0,0])
dh_parameters = [[a[0],alpha[0],d[0],theta[0]], [a[1],alpha[1],d[1],theta[1]], [a[2],alpha[2],d[2],theta[2]], [a[3],alpha[3],d[3],theta[3]], [a[4],alpha[4],d[4],theta[4]], [a[5],alpha[5],d[5],theta[5]], [a[6],alpha[6],d[6],theta[6]]]
dh_parameters = np.matrix(dh_parameters)
print(dh_parameters)

[[ 0.00000000e+00 -1.57079633e+00  3.40000000e+02  5.23598776e-01]
 [ 0.00000000e+00  1.57079633e+00  0.00000000e+00 -7.85398163e-01]
 [ 0.00000000e+00  1.57079633e+00  4.00000000e+02  1.04719755e+00]
 [ 0.00000000e+00 -1.57079633e+00  0.00000000e+00  1.30899694e+00]
 [ 0.00000000e+00 -1.57079633e+00  4.00000000e+02 -3.49065850e-01]
 [ 0.00000000e+00  1.57079633e+00  0.00000000e+00  1.65806279e+00]
 [ 0.00000000e+00  0.00000000e+00  1.26000000e+02 -1.39626340e+00]]


"DH transformation matrix" function        


In [0]:
def DH(angles, joint):
     #angles = np.rad2deg(angles)
     DH = [[cos(angles[joint]), -(sin(angles[joint])*cos(alpha[joint])),  sin(angles[joint])*sin(alpha[joint]),   a[joint]*cos(angles[joint])], 
             [sin(angles[joint]),  cos(angles[joint])*cos(alpha[joint]),  -(cos(angles[joint])*(sin(alpha[joint]))), a[joint]*sin(angles[joint])],
             [0,                   sin(alpha[joint]),               cos(alpha[joint]),                d[joint]],
             [0,                         0,                          0,                          1]]
     DH = np.matrix(DH)
     return DH


"Transformation chain"  function

In [0]:
def transformation(angles,joints):
  temp3 = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
  temp3 = np.matrix(temp3)
  for  i in range(joints):
    temp3 = dot(temp3,DH(angles, i))
  return temp3

Global Configuration

In [0]:
def gc(i):            #theta2 is gc(1); theta4 is gc(3); theta6 is gc(5)
  if(theta[i]>= 0): 
    return  1
  else:
    return -1

 FORWARD KINEMATICS  _  
 

---


 0 to 7 - base to end effector transformation matrix _

In [0]:
print(transformation(theta, 7))

[[ 9.43485717e-01 -1.01655600e-01 -3.15437537e-01 -2.99089573e+02]
 [ 8.96564010e-02  9.94595417e-01 -5.23611126e-02 -5.42700415e+02]
 [ 3.19055529e-01  2.11209675e-02  9.47500646e-01  6.78830334e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [0]:
T07 = transformation(theta, 7)
R07 = matrix(T07[:3,:3])
P07 = matrix(T07[:3,3])

P02 = matrix([0,0,d_bs]).T
P24 = matrix([0,d_se,0]).T
P46 = matrix([0,0,d_ew]).T
P67 = matrix([0,0,d_wf]).T

P26 = P07 - P02 - dot(R07,P67)
print(P26)

[[-259.34444334]
 [-536.10291469]
 [ 219.44525285]]


virtual theta 4

In [0]:
def mag(x):
    return math.sqrt(sum(i**2 for i in x))

In [0]:
print(P26)
mag(P26)

[[-259.34444334]
 [-536.10291469]
 [ 219.44525285]]


634.6826722329882

In [0]:
k = (mag(P26)**2 - d_se**2 - d_ew**2)/(2*d_se*d_ew)
theta4_v = gc(3)*acos(k)

virtual theta 1

In [0]:
R01 = DH(theta, 0)[:3,2:3]
print(R01)
alignment =mag(np.cross(P26, R01, axis=0))  #axis = 0 because R01 and P26 are column vectors
print(alignment)

[[-5.00000000e-01]
 [ 8.66025404e-01]
 [ 6.12323400e-17]]
539.3149081990414


In [0]:
if alignment > 0:
  theta1_v = math.atan2(P26[1:2], P26[0:1])
elif alignment == 0:
  theta1_v = 0

calculating Phi for Virtual Theta 2 

In [0]:
phi = math.acos((d_se**2 + (mag(P26))**2 - d_ew**2) / ( 2*d_se*(mag(P26))))
print(phi)
theta2_v = math.atan2(math.sqrt( (P26[0:1])**2+(P26[1:2])**2 ), P26[2:3]) + gc(3)*phi
print(theta2_v)

0.6544984694978733
1.8722505889397933


Theta Virtual 3, this angle is kept zero so we can convert 7dof robot into a 6 dof robot

In [0]:
theta3_v = 0

virtual angles

---


ASSEMBLE

In [0]:
theta_v = np.array([theta1_v, theta2_v, theta3_v, theta4_v])

Psi angle, the angle between the "virtual elbow" and "real elbow" wrt shoulder-wrist axis

In [0]:
P02_v = transformation(theta_v,2)[0:3, 3:4]
P02 = transformation(theta, 2)[0:3,3:4]

P04_v = transformation(theta_v,4)[0:3, 3:4]
P04 = transformation(theta, 4)[0:3, 3:4]

P06 = transformation(theta,6)[0:3, 3:4]
P06_v = P06

In [0]:

j = P04_v - P02_v
q = P06_v - P02_v
normal_virtual_sew = np.cross( ( (j) / mag(j) ) , ( (q) / mag(q) ) , axis = 0 )

j = P04 - P02
q = P06 - P02
normal_real_sew = np.cross( ( (j) / mag(j) ) , ( (q) / mag(q) ) , axis = 0 )
print((j)/mag(j))
print(q/mag(q))
print(normal_real_sew)

[[-0.61237244]
 [-0.35355339]
 [ 0.70710678]]
[[-0.40862065]
 [-0.84467867]
 [ 0.34575586]]
[[ 0.47503486]
 [-0.07720707]
 [ 0.37278872]]


In [0]:
mod_real_sew = np.linalg.norm(normal_real_sew)
mod_virtual_sew = np.linalg.norm(normal_virtual_sew)

if mod_real_sew == 0:
  mod_real_sew = 0.0000000000000001
if mod_virtual_sew == 0:
  mod_virtual_sew = 0.0000000000000001

In [0]:

normal_virtual_sew_hat = normal_virtual_sew  /  mod_virtual_sew
normal_real_sew_hat = normal_real_sew  /  mod_real_sew
print(normal_real_sew)

[[ 0.47503486]
 [-0.07720707]
 [ 0.37278872]]


In [0]:


g =np.cross((normal_virtual_sew_hat),(normal_real_sew_hat), axis = 0)
g= dot(g.T, P26)

if g>= 0:
  g = 1
elif g < 0:
  g = -1

psi = g* math.acos(dot(normal_virtual_sew_hat.T, normal_real_sew_hat))
print(psi)

-2.43055109252351


INVERSE KINEMATICS

In [0]:
def skew(x):  
    return np.matrix([[0, -x.item(2), x.item(1)],
                     [x.item(2), 0, -x.item(0)],
                     [-x.item(1), x.item(0), 0]])

In [0]:
P26_hat = P26/mag(P26)

In [0]:
P26_skew = skew(P26_hat)

R03_v = transformation(theta_v, 2)[0:3,0:3]

As = np.matmul(P26_skew, R03_v)
Bs = -1 * np.matmul( (P26_skew**2),  R03_v)
Cs = np.matmul((np.matmul(P26_hat, P26_hat.T)), R03_v) 

Obtaining R03 for theta_real1, theta_real2, theta_real3

In [0]:
R03 = As*sin(psi) + Bs*cos(psi) + Cs

In [0]:
As[2,1]*sin(psi) + Bs[2,1]*cos(psi) + Cs[2,1]

-0.6123724356957941

In [0]:
theta1_r = math.atan2( gc(1) * (As[1,1]*sin(psi) + Bs[1,1]*cos(psi) + Cs[1,1]),  gc(1) * (As[0,1]*sin(psi) + Bs[0,1]*cos(psi) + Cs[0,1])  )
theta2_r = gc(1)* math.acos(As[2,1]*sin(psi) + Bs[2,1]*cos(psi) + Cs[2,1])
theta3_r = math.atan2( gc(1) * (-As[2,2]*sin(psi) - Bs[2,2]*cos(psi) - Cs[2,2]),  gc(1) * (-As[2,0]*sin(psi) - Bs[2,0]*cos(psi) - Cs[2,0])  )
theta4_r = theta4_v

REAL ANGLES

---
ASSEMBLE


In [0]:
theta_r = np.array([theta1_r, theta2_r, theta3_r, theta4_r])

In [0]:
R34 = DH(theta_r, 3)[0:3, 0:3]

Aw = R34.T * As.T * R07
Bw = R34.T * Bs.T * R07
Cw = R34.T * Cs.T * R07

In [0]:
R47 = Aw*sin(psi) + Bw*cos(psi) + Cw

theta5_r = math.atan2( (gc(5)*( Aw[1,2]*sin(psi) + Bw[1,2]*cos(psi) + Cw[1,2]))  , (gc(5)*( Aw[0,2]*sin(psi) + Bw[0,2]*cos(psi) + Cw[0,2]))  )
theta6_r = gc(5)*math.acos( Aw[2,2]*sin(psi) + Bw[2,2]*cos(psi) + Cw[2,2] )
theta7_r = math.atan2( (gc(5)*( Aw[2,1]*sin(psi) + Bw[2,1]*cos(psi) + Cw[2,1]))  , (gc(5)*( - Aw[2,0]*sin(psi) - Bw[2,0]*cos(psi) - Cw[2,0]))  )

theta_r = np.append(theta_r, theta5_r)
theta_r = np.append(theta_r, theta6_r)
theta_r = np.append(theta_r, theta7_r)
print(theta_r)
print(np.rad2deg(theta_r))

[-0.16112043 -2.22985436  1.10714872  1.30899694 -1.84143359  1.98616917
 -1.2230364 ]
[  -9.23152048 -127.76124391   63.43494882   75.         -105.50637281
  113.79911059  -70.07482377]


Testing the obtained real angles after INVERSE KINEMATICS

In [0]:
# "theta_r" contains obtained real angles after IK
print(transformation(theta_r, 7))

[[ 9.43485717e-01 -1.01655600e-01 -3.15437537e-01 -3.83660889e+02]
 [ 8.96564010e-02  9.94595417e-01 -5.23611126e-02 -3.00815881e+02]
 [ 3.19055529e-01  2.11209675e-02  9.47500646e-01  1.44361071e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
